# 12. Improving Simple CNN

In [1]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim

import torchvision.utils
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import numpy as np
import os

## 12.1 Preparing Custom Data

In [2]:
img_dir = "./data/jamo"
img_data = dsets.ImageFolder(img_dir, transforms.Compose([
            transforms.Grayscale(),
# https://pytorch.org/docs/stable/torchvision/transforms.html
#           Data Augmentation
#           transforms.RandomRotation(15)
#           transforms.CenterCrop(28),
#           transforms.Lambda(lambda x: x.rotate(15)),
    
#           Data Nomalization
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.5,), std=(0.5,))
# Normalize a tensor image with mean and standard deviation.
# Given mean: (M1,...,Mn) and std: (S1,..,Sn) for n channels,
# this transform will normalize each channel of the input torch.
# *Tensor i.e. input[channel] = (input[channel] - mean[channel]) / std[channel]
            ]))

#https://pytorch.org/docs/stable/torchvision/transforms.html

print(img_data.classes)
print(img_data.class_to_idx)

['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ', 'ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
{'ㄱ': 0, 'ㄲ': 1, 'ㄴ': 2, 'ㄷ': 3, 'ㄸ': 4, 'ㄹ': 5, 'ㅁ': 6, 'ㅂ': 7, 'ㅃ': 8, 'ㅅ': 9, 'ㅆ': 10, 'ㅇ': 11, 'ㅈ': 12, 'ㅉ': 13, 'ㅊ': 14, 'ㅋ': 15, 'ㅌ': 16, 'ㅍ': 17, 'ㅎ': 18, 'ㅏ': 19, 'ㅐ': 20, 'ㅑ': 21, 'ㅒ': 22, 'ㅓ': 23, 'ㅔ': 24, 'ㅕ': 25, 'ㅖ': 26, 'ㅗ': 27, 'ㅘ': 28, 'ㅙ': 29, 'ㅛ': 30, 'ㅜ': 31, 'ㅝ': 32, 'ㅞ': 33, 'ㅟ': 34, 'ㅠ': 35, 'ㅡ': 36, 'ㅢ': 37, 'ㅣ': 38}


In [3]:
batch_size = 100

In [4]:
def train_test_split(data, train_ratio, batch_size, stratify) :
    
    length = len(data)
    
    cut = int(len(data)*train_ratio)
    train_indices = np.random.shuffle(np.random.permutation(np.arange(length))[:cut])
    test_indices = np.random.shuffle(np.random.permutation(np.arange(length))[cut:])
        
    if stratify :
        
        count = [0]*len(img_data.classes)
        for _, label in img_data :
            count[label] += 1

        weight = []    
        for i, (_, label) in enumerate(img_data) :
            weight.append(1/ count[label])
        weight = np.array(weight)
        
        train_indices = np.random.choice(length, cut, p=weight/sum(weight), replace=False)
        test_indices = np.array(list(set(range(length)) - set(train_indices)))
        
    train_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=False, sampler = torch.utils.data.SubsetRandomSampler(train_indices), drop_last = True)
    test_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=False, sampler = torch.utils.data.SubsetRandomSampler(test_indices), drop_last = True)

    return train_loader, test_loader, len(train_indices), len(test_indices)

In [5]:
train_loader, test_loader, train_num, test_num = train_test_split(img_data, 0.8, batch_size, True)

## 12.2 Define Model

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer = nn.Sequential(
            nn.Conv2d(1,16,5),
            nn.ReLU(),
            #Dropout
            nn.Dropout2d(0.5),
            nn.Conv2d(16,32,5),
            #Batch Nomalization
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,64,5),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        
        self.fc_layer = nn.Sequential(
            nn.Linear(64*5*5,500),
            nn.ReLU(),
            nn.Linear(500,39)
        )
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
#                 Weight Initialization
#                 init.xavier_normal(m.weight.data)
                init.kaiming_normal_(m.weight.data)
                m.bias.data.fill_(0)
        
            elif isinstance(m, nn.Linear):
                init.kaiming_normal_(m.weight.data)
                m.bias.data.fill_(0)                
        
    def forward(self,x):
        out = self.layer(x)
        out = out.view(batch_size,-1)
        out = self.fc_layer(out)

        return out

model = CNN().cuda()

In [7]:
loss = nn.CrossEntropyLoss() # Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Momentum & Weight Regularization(L2)
# optim.SGD(model.parameters(), lr=1e-2, momentum=0.9, weight_decay=1e-5)

In [8]:
num_epochs = 50

In [9]:
def test_model() :
    
    model.eval()
    
    correct = 0
    total = 0

    for images, labels in test_loader:

        images = images.cuda()
        outputs = model(images)

        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels.cuda()).sum()

    print('Accuracy of test images: %f %%' % (100 * float(correct) / total))

In [10]:
# Learning Rate Scheduler
import torch.optim.lr_scheduler as lr_scheduler
#scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma= 0.99)
#scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[10,30,80], gamma= 0.1)
scheduler = lr_scheduler.ExponentialLR(optimizer, gamma= 0.99)
#scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min')

for epoch in range(num_epochs):
    
    scheduler.step()
    
    for i, (batch_images, batch_labels) in enumerate(train_loader):
        
        X = batch_images.cuda()
        Y = batch_labels.cuda()

        pre = model(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, train_num//batch_size, cost.item()))
            test_model()
            model.train()

Epoch [1/50], lter [100/224], Loss: 0.6927
Accuracy of test images: 78.410714 %
Epoch [1/50], lter [200/224], Loss: 0.5393
Accuracy of test images: 85.928571 %
Epoch [2/50], lter [100/224], Loss: 0.3914
Accuracy of test images: 87.732143 %
Epoch [2/50], lter [200/224], Loss: 0.2195
Accuracy of test images: 93.517857 %
Epoch [3/50], lter [100/224], Loss: 0.1025
Accuracy of test images: 92.589286 %
Epoch [3/50], lter [200/224], Loss: 0.3041
Accuracy of test images: 93.875000 %
Epoch [4/50], lter [100/224], Loss: 0.1127
Accuracy of test images: 95.464286 %
Epoch [4/50], lter [200/224], Loss: 0.1038
Accuracy of test images: 96.071429 %
Epoch [5/50], lter [100/224], Loss: 0.0819
Accuracy of test images: 95.660714 %
Epoch [5/50], lter [200/224], Loss: 0.0966
Accuracy of test images: 96.232143 %
Epoch [6/50], lter [100/224], Loss: 0.1411
Accuracy of test images: 93.625000 %
Epoch [6/50], lter [200/224], Loss: 0.0526
Accuracy of test images: 96.142857 %
Epoch [7/50], lter [100/224], Loss: 0.09